<a href="https://colab.research.google.com/github/ap-nlp-research/language_translation_en_ru_tf2/blob/master/MachineTranslationModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation Project


The goal of the project is to compare the strength of the following recurrent models:

1. Embedded GRU
2. Embedded Bidirectional GRU
3. Embedded GRU encoder-decoder model
4. Embedded GRU encoder-decoder model with Multiplicative Attention

The models implemented in Tensorflow 2.0 with Keras as a high-level API. Models are trained and analyzed based on [TedHrlrTranslate dataset](https://www.tensorflow.org/datasets/datasets#ted_hrlr_translate).

In [2]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
import os
import subprocess
import re
import numpy as np
from functools import partial
import tensorflow as tf
from tensorflow import keras

## Data ETL

The data load, extraction, and transformation is done with data_etl() method. This method returns a dictionary containing source data stored under 'x' label. Target data is stored under 'y' label. In addition to the source and target data, the dictionary contains x and y tockenizers (stored as 'x_tk' and 'y_tk'):

In [0]:
def data_etl(download_dir: str = ".", file_name: str = "en_ru.tgz", n_lines: int = 3000,
             lenght_lim_lower: int = 4, length_lim_upper: int = 500, num_words: int = 5000) -> dict:
    path = os.path.join(download_dir, file_name)

    print("Start data ETL")

    if os.path.isfile(path):
        print("Reuse pre-downloaded " + path)
        # extract
        subprocess.run("tar --extract --file".split() + [path])
    else:
        # download a dataset
        print("Start downloading")
        completed = subprocess.run(
                ["curl",
                 "https://s3.amazonaws.com/web-language-models/paracrawl/release1/paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz",
                 "--output", 
                 path]
        )
        if completed.check_returncode() is not None:
            print("Downloading returned error code:", completed.check_returncode())
    # extract
    subprocess.run("tar --extract --file".split() + [path])
    print("Data downloaded and extracted")

    # read data
    en = read_lines("paracrawl-release1.en-ru.zipporah0-dedup-clean.en", n_lines)
    ru = read_lines("paracrawl-release1.en-ru.zipporah0-dedup-clean.ru", n_lines)

    print("Extracted successfully")

    # filter out small samples
    indices = list(filter(lambda idx: length_lim_upper > len(en[idx].split()) >=lenght_lim_lower, range(len(en))))
    en = [en[idx] for idx in indices]
    ru = [ru[idx] for idx in indices]

    # Tockenize
    en = [re.sub("[0-9]", " \g<0>", s) for s in en]
    ru = [re.sub("[0-9]", " \g<0>", s) for s in ru]

    x, x_tk = tokenize(en, num_words=num_words, filters_regex=None)
    y, y_tk = tokenize(ru, num_words=num_words * 3, filters_regex=None)

    x = pad(x)
    y = pad(y)

    print("Transformed successfully")

    return {'x': x, 'y': y, 'x_tk': x_tk, 'y_tk': y_tk}


def read_lines(from_file: str, n_lines: int) -> list:
    if not os.path.isfile(from_file):
        raise FileNotFoundError(from_file)

    lines: list = []
    with open(from_file, "r") as f:
        for idx, line in enumerate(f):
            if idx < n_lines:
                lines.append(line)
            else:
                break

    return lines


def tokenize(x: list, num_words: int = 5000, filters_regex: str = None):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :param num_words: maximum number of words in the vocabulary - rare words will be replaced with a [rare] token
    :param filters_regex: regular expression filter string such as [^a-zA-Z0-9/-]
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    if filters_regex is not None:
        x_tk = keras.preprocessing.text.Tokenizer(num_words=num_words, filters=filters_regex, oov_token="[rare]")
    else:
        x_tk = keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token="[rare]")

    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


def pad(x, length=None) -> np.ndarray:
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])

    return keras.preprocessing.sequence.pad_sequences(x, maxlen=length, padding='post')

dataset = data_etl()

{
    'x': np.ndarray,
    'y': np.ndarray,
    'x_tk': keras.preprocessing.text.Tokenizer,
    'y_tk': keras.preprocessing.text.Tokenizer
}

In [5]:
dataset = data_etl(download_dir=".", n_lines=5000, num_words=3000)

Start data ETL
Reuse pre-downloaded ./en_ru.tgz
Data downloaded and extracted
Extracted successfully
Transformed successfully


## Utility Functions

In addition to the data ETL, the code below provides two additional functions for converting logits into word indicies and converting word indicies into text.

In [0]:
def logits_to_id(logits):
    """
    Turns logits into word ids
    :param logits: Logits from a neural network
    """
    return [prediction for prediction in np.argmax(logits, 1)]

def id_to_text(idx, tokenizer):
    """
    Turns id into text using the tokenizer
    :param idx: word id
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in idx]).replace(" <PAD>", "")

In [7]:
print("Here is an example for a samples number 1:")
print("Source('en') example:", id_to_text(dataset['x'][0], dataset['x_tk']))
print("Target('ru') example:", id_to_text(dataset['y'][0], dataset['y_tk']))
print(" ")
print("Samples number 2:")
print("Source('en') example:", id_to_text(dataset['x'][1], dataset['x_tk']))
print("Target('ru') example:", id_to_text(dataset['y'][1], dataset['y_tk']))
print("source vocabulary size:", dataset['x'].max())
print("target vocabulary size:", dataset['y'].max())
print("Source shape:", dataset['x'].shape)
print("Target shape:", dataset['y'].shape)

Here is an example for a samples number 1:
Source('en') example: the company has been registered with the [rare] court in prague in section b file 1 4 8 5 7
Target('ru') example: фирма зарегистрирована в городском суде в г праге раздел б вкладыш 1 4 8 5 7
 
Samples number 2:
Source('en') example: since january 2 0 0 9 new office premises in the street v [rare] 3 prague 1 have been opened these premises are situated in a close [rare] of all our partner banks
Target('ru') example: с января месяца 2 0 0 9 г открыты новые офисные помещения на улице v kolkovně 3 в колковне д 3 прага 1 данные помещения находятся поблизости всех банков партнеров
source vocabulary size: 2999
target vocabulary size: 8999
Source shape: (3603, 399)
Target shape: (3603, 407)


## Models

The models are implemented with a similar set of parameters. The main idea is to keep models as small and simple as possible to quickly train them and validate the difference the primarely derived from model architectures. The summary of main hyper parameters presented below:

* Mapping:
    - Embeddings - word indices will be mapped into a 16-dimentional space
    - Dense mapping - recurrence outputs mapped into the target-language space, represented with OHE, via Dense mapping
* Layers:
    - GRU - number of units 128
    - Bidirectional GRU - number of untis set up to 64 in order to keep the total number of units the same (128)
    - Batch Normalization - To speed up the training batch normalization is inserted after embeddings and before dense mapping
* Optimization:
    - Adam - all models trained with Adam optimizer and the same learning rate (1e-3)
* Loss function:
    - sparse_categorical_crossentropy_from_logits - keras.losses.sparse_categorical_crossentropy

In [0]:
learning_rate = 1e-3
embeddings_units = 16
gru_units = 128
epochs = 2
validation_split = 0.05
batch_size = 128
sparse_categorical_crossentropy_from_logits = partial(keras.losses.sparse_categorical_crossentropy, from_logits=True)

**Model list:**

1. Embedded GRU
2. Embedded Bidirectional GRU
3. Embedded GRU encoder-decoder model
4. Embedded GRU encoder-decoder model with Multiplicative Attention

#### Model 1 - Embedded GRU

In [0]:
def embedded_gru_model(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_seq = keras.Input(input_shape[1:])
    embedded_seq = keras.layers.Embedding(source_vocab_size, embeddings_units, input_length=output_sequence_length)(input_seq)
    rnn = keras.layers.GRU(gru_units, return_sequences=True)(embedded_seq)
    logits = keras.layers.TimeDistributed(keras.layers.Dense(target_vocab_size))(rnn)
    model = keras.Model(input_seq, logits)
    model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate, clipnorm=1.0),
                  metrics=['accuracy'])
    return model


max_length = max(dataset['x'].shape[1], dataset['y'].shape[1])
x = keras.layers.ZeroPadding1D((0, max_length-dataset['x'].shape[1]))(dataset['x'][:,:,None])
x = keras.backend.squeeze(x, -1)
y = keras.layers.ZeroPadding1D((0, max_length-dataset['y'].shape[1]))(dataset['y'][:,:,None])
print("Source shape:", x.shape)
print("Target shape:", x.shape)

  
# Train the neural network
embed_rnn_model = embedded_gru_model(
    x.shape,
    y.shape[1],
    len(dataset['x_tk'].word_index)+1,
    len(dataset['y_tk'].word_index)+1)



print("Model summary:")
embed_rnn_model.summary()
embed_rnn_model.fit(x, 
                    y, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_split=validation_split)

Source shape: (3603, 407)
Target shape: (3603, 407)
Model summary:
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 407)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 407, 16)           134400    
_________________________________________________________________
unified_gru_1 (UnifiedGRU)   (None, 407, 128)          56064     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 407, 14834)        1913586   
Total params: 2,104,050
Trainable params: 2,104,050
Non-trainable params: 0
_________________________________________________________________
Train on 3422 samples, validate on 181 samples
Epoch 1/2
 768/3422 [=====>........................] - ETA: 11:29 - loss: 5.2484 - accuracy: 0.6369

In [0]:
# Print prediction(s)
sentense_id = -1
print("Source('en') example:", id_to_text(dataset['x'][sentense_id], dataset['x_tk']))
print("Target('ru') example:", id_to_text(dataset['y'][sentense_id], dataset['y_tk']))
print("Translation(en_ru) example:", logits_to_text(logits_to_id(embed_rnn_model.predict(dataset['x'][sentense_id][None, :])[0]), dataset['y_tk']))